# Conjugate Gradients on Models

Suppose we have some cost function we want to optimize to determine parameters $w$:
\begin{equation}
\min_w g(w)
\end{equation}
This is the same problem as 
\begin{equation}
\frac{\partial}{\partial w} g(w) = 0
\end{equation}
If $g$ is the loss function of a neural network or anything with a machine-learn-y name, people are just going to use stochastic gradient descent, which is naive but works well.

If the model is a linear system of equations, then its equivalent cost function is
\begin{equation}
g(w) = \frac{1}{2} w^T\mathbf{A}w-w^T\mathbf{b}
\end{equation}
The loss function's gradient is the system of equation $\mathbf{A}w-\mathbf{b}=0$, and one-step of Newton's method would involve just solving that same equation.

## The method


- $r := \frac{\partial}{\partial w} g(w_k)$
- $z := r$
- Do until convergence:
  - $\alpha := r\cdot r \,/\, z\cdot H z$
  - $w += \alpha z$
  - $r_{new} := \frac{\partial}{\partial w} g(w_k)$
  - $\beta = r_{new} \cdot r_{new}\,/\,r_{old}\cdot r_{old}$
  - $ z_{new} = r_{new} + \beta z$
  
## The Action of the Hessian

We can compute the Hessian for arbitrary models (look in [operations.py](operations.py)`:NewtonsMethod()`), but this suffers from two problems:

- it will be huge and dense, $N_{param}\times N_{param}$
- generating the graph can blow up.

Just dealing the Hessian is one issue with naively doing more advanced methods. Then, for arbitrary models with unneccessary parameters, the Hessian itself is likely rank defficient and cannot be used for the direct application of Newton's method. However, it turns out that we don't need the Hessian, what we need is its linear action on an arbitrary vector:
\begin{equation}
H \mathbf{z}
\end{equation}

We can avoid assembling a large dense matrix by remembering the interchangability of derivatives:
\begin{equation}
\left[\frac{\partial^2 g}{\partial w \partial w}\right] \mathbf{z} = \frac{\partial}{\partial x} \left(\frac{\partial g}{\partial x} \cdot \mathbf{z} \right)
\end{equation}


## Implementing it!

Let's do a proof of concept with a tiny linear fit:

In [1]:
import tensorflow as tf
import numpy as np
from operations import CatVariable, flatpack
sess = tf.InteractiveSession()

In [2]:
a = tf.Variable(tf.truncated_normal(shape=(1,1)))
b = tf.Variable(tf.truncated_normal(shape=(1,)))
x = tf.constant([0,1],dtype=tf.float32)
y_true = tf.constant([[2,5]],dtype=tf.float32)
y_pred = a*x+b
goal = tf.losses.mean_squared_error( y_true,y_pred)
params = [a, b]

In [3]:
mvmul = lambda K,x : tf.einsum('ij,i->j',K,x)

In [4]:
dot = lambda a,b : tf.reduce_sum([ tf.tensordot(p_,z_, (range(len(p_.shape)),range(len(z_.shape))) )
                                   for p_,z_ in zip(a,b)])

In [5]:
w = tf.Variable(tf.truncated_normal(shape=(2,)))
A = tf.constant([[1.0,2.0],[2.0,3.0]])
b = tf.constant([3.0,3.0]) # hashtag life-hack to get the answer
# goal = tf.einsum('i,i->',w,0.5*A*w-b)
goal = tf.tensordot(w,0.5*mvmul(A,w)-b, axes=1)
params = [ w ]

Steepest descent is just this operation:

In [36]:
steep_step = [ tf.assign(w_, w_-1.0e-3*g_) for w_,g_ in zip(params,grads) ]

## No depdendencies:

The variables we need:

In [6]:
z_params = [tf.Variable(p_) for p_ in params] # The conjugates
r_params = [tf.Variable(p_) for p_ in params] # A temp to store gradients
alpha = tf.Variable(1.0)
beta = tf.Variable(1.0)
r_dot_r_new = tf.Variable(1.0)
r_dot_r_old = tf.Variable(1.0)

In [7]:
grads = tf.gradients(goal, params)
z_dot_grad = dot(z_params,grads)
Hz = tf.gradients(z_dot_grad,params)
r_dot_r = dot(r_params,r_params)
z_dot_Hz = dot(z_params,Hz)

Test to make sure we can extract out the values of A we expect:

In [37]:
sess.run([tf.assign(z_params[0],[0.0,1.0]),Hz])

[array([0., 1.], dtype=float32), [array([2., 3.], dtype=float32)]]

In [8]:
reset_z    = [ tf.assign(z_,           -g_) for z_,g_ in zip(z_params,grads) ]
calc_grad  = [ tf.assign(r_,           -g_) for r_,g_ in zip(r_params,grads) ]
calc_r_dot_r=[ tf.assign( r_dot_r_old, r_dot_r ) ]
calc_alpha = [ tf.assign(alpha,        r_dot_r_old/z_dot_Hz) ]
calc_newp  = [ tf.assign(p_,           p_+alpha*z_) for p_,z_ in zip(params,z_params)  ]
calc_beta  = [ tf.assign(beta,         r_dot_r/r_dot_r_old) ]
calc_z     = [ tf.assign(z_,           r_+beta*z_) for z_,r_ in zip(z_params,r_params) ]

cg_first_no_flow = reset_z + calc_grad
cg_step_no_flow = calc_r_dot_r + calc_alpha + calc_newp + calc_grad + calc_beta + calc_z

Because that program listing has assignments, we have to do the sess.runs() one at a time:

In [10]:
sess.run(tf.global_variables_initializer())
sess.run(tf.assign(w,[-3.00,2.001]))
print(sess.run(cg_first_no_flow))
print(sess.run([goal,params,Hz]))
for i in range(3):
    for s in cg_step_no_flow:
        print(sess.run(s))
    print(sess.run([goal,alpha,beta,params,grads]))

[array([1.9980001, 2.9970002], dtype=float32), array([1.9980001, 2.9970002], dtype=float32)]
[1.4970016, [array([-3.   ,  2.001], dtype=float32)], [array([ 7.9920006, 12.987001 ], dtype=float32)]]
12.974014
0.23636362
[-2.5277455  2.7093818]
[ 0.10898185 -0.07265425]
0.0013223111
[ 0.11162382 -0.06869128]
[-0.036291122, 0.23636362, 0.0013223111, [array([-2.5277455,  2.7093818], dtype=float32)], [array([-0.10898185,  0.07265425], dtype=float32)]]
0.017155683
-4.230769
[-3.         2.9999988]
[2.3841858e-06 3.8146973e-06]
1.1795658e-09
[2.3843174e-06 3.8146163e-06]
[0.0, -4.230769, 1.1795658e-09, [array([-3.       ,  2.9999988], dtype=float32)], [array([-2.3841858e-06, -3.8146973e-06], dtype=float32)]]
2.0236257e-11
0.23607424
[-2.9999995  2.9999998]
[-0. -0.]
0.0
[0. 0.]
[-4.7683716e-07, 0.23607424, 0.0, [array([-2.9999995,  2.9999998], dtype=float32)], [array([0., 0.], dtype=float32)]]


## With control flow:

Now, when we include control flow to try to get *one operation* that tensorflow can execute, it turns into a complete mess:

In [11]:
grads = tf.gradients(goal, params)
reset_z            = [ tf.assign(z_,           -g_) for z_,g_ in zip(z_params,grads) ]
calc_grad_initial  = [ tf.assign(r_,           -g_) for r_,g_ in zip(r_params,grads) ]
z_dot_grad = dot(z_params,grads)
Hz = tf.gradients(z_dot_grad,params)
z_dot_Hz = dot(z_params,Hz)
r_dot_r = dot(r_params,r_params)
calc_r_dot_r=[ tf.assign( r_dot_r_old, r_dot_r ) ]
with tf.control_dependencies(calc_r_dot_r):
    calc_alpha = [ tf.assign(alpha,        r_dot_r_old/z_dot_Hz) ]
    with tf.control_dependencies(calc_alpha):
        calc_newp  = [ tf.assign(p_,           p_ + alpha*z_) for p_,z_ in zip(params,z_params)  ]
        with tf.control_dependencies(calc_newp):
            goal2 = tf.tensordot(w,0.5*mvmul(A,w)-b, axes=1)
            grads = tf.gradients(goal2, params)

            calc_grad_during  = [ tf.assign(r_,    -g_) for r_,g_ in zip(r_params,grads) ]
            with tf.control_dependencies(calc_grad_during):
                r_dot_r = dot(r_params,r_params)
                calc_beta  = [ tf.assign(beta,         r_dot_r/r_dot_r_old) ]
                with tf.control_dependencies(calc_beta):
                    calc_z = [ tf.assign(z_,           r_ + beta*z_) for z_,r_ in zip(z_params,r_params) ]
cg_first = reset_z + calc_grad_initial
cg_step = calc_r_dot_r + calc_alpha + calc_newp + calc_grad_during + calc_beta + calc_z
cg_step_1 = calc_r_dot_r + calc_alpha + calc_newp
cg_step_2 = calc_grad_during + calc_beta + calc_z

In [38]:
sess.run([tf.assign(z_params[0],[1.0,0.0]),z_dot_Hz])

[array([1., 0.], dtype=float32), 1.0]

We can try to run it the same way, but for some reason including the control flow directives **break everything**

In [13]:
sess.run(tf.global_variables_initializer())
sess.run(tf.assign(w,[-3.00,2.001]))
print(sess.run(cg_first))
print(sess.run([goal,params,Hz]))
for i in range(5):
    for s in cg_step:
        print(sess.run(s))
    print(sess.run([goal,alpha,beta,params,grads]))

[array([1.9980001, 2.9970002], dtype=float32), array([1.9980001, 2.9970002], dtype=float32)]
[1.4970016, [array([-3.   ,  2.001], dtype=float32)], [array([ 7.9920006, 12.987001 ], dtype=float32)]]
12.974014
0.23636362
[-2.5277455  2.7093818]
[-1.7800364 -3.1423092]
4.0119076
[6.7981625 8.535282 ]
[55.87135, 0.95380926, 9.056986, [array([ 6.808075, 15.127918], dtype=float32)], [array([34.06391 , 55.999905], dtype=float32)]]
473.91452
0.95380926
[13.292225 23.26895 ]
[ -79.59634 -130.78268]
229.29004
[ 55288.406 -30936.125]
[29339468000.0, -272.27548, 46288.555, [array([-14979802.,   8515906.], dtype=float32)], [array([ 2052007., -4411889.], dtype=float32)]]
248777240000.0
-272.27548
[-30033480.  16939054.]
[-5637245. 14087707.]
99656.84
[3.8029670e+14 1.4090865e+14]
[-2.8811114e+29, 0.54103243, 9868888000.0, [array([-1.1826901e+15,  8.5312680e+14], dtype=float32)], [array([5.2356349e+14, 1.9400018e+14], dtype=float32)]]
2.2644356e+29
0.54103243
[-9.7693729e+14  9.2936294e+14]
[-1.240013

After six hours of trying to implement a very simple program, I give up. TensorFlow is a great language for describing model architectures, but a horrendously poor languagefor describing algorithms. We shouldn't try to implement anything more complicated than gradient methods in TF. Maybe tf2 will be better.

The expected answer:

In [20]:
nA,nb=sess.run([A,b])

In [21]:
np.linalg.solve(nA,nb)

array([-3.,  3.], dtype=float32)

In [22]:
nA.dot([-3,3])

array([3., 3.])